In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import shutil
import numpy as np
import tensorflow as tf


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4932948616509810365
]


In [4]:
from yolov3.dataset_1 import Dataset


<function yolov3.yolov4.compute_loss1(pred, conv, label, bboxes, i=0, CLASSES='model_data/coco/coco.names')>

In [8]:
"""
https://fixexception.com/tensorflow/the-first-argument-to-layer-call-must-always-be-passed/
"""
import tensorflow as tf 

#Defining model with Layers
def model_arch(n_features: int):
    i = tf.keras.layers.Input(shape=(n_features,))

    hidden_dense = tf.keras.layers.Dense(64)(i)
    hidden_dense = tf.keras.layers.BatchNormalization()(hidden_dense)
    hidden_dense = tf.keras.layers.Activation(tf.nn.tanh)(hidden_dense)

    o = tf.keras.layers.Dense(1)(hidden_dense)
    o = tf.keras.layers.BatchNormalization()(o)
    o = tf.keras.layers.Activation("sigmoid")(o)

    classifier = tf.keras.models.Model(inputs=i, outputs=o)

    opt = tf.keras.optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
    classifier.compile(
        loss="binary_crossentropy",
        optimizer=opt,
        metrics=["accuracy"],
    )
    
    return classifier

clf = model_arch(4)

input = tf.keras.layers.Input(shape=(4,))
model=clf(input) #<--- passing an input

type(model)

tensorflow.python.keras.engine.keras_tensor.KerasTensor